In [1]:
%matplotlib notebook

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import os 
from tqdm import tqdm

import multiprocessing as mp

import time 
from natsort import natsorted
import moviepy.video.io.ImageSequenceClip

import copy 


In [44]:
sys.setrecursionlimit(2000) 

In [3]:
from simulation import leaf, QuadTree
from visualizer import renderParticles, drawTree, plotter

In [4]:
plt.ioff()

In [284]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import multiprocessing as mp

        
def quadAssign(p, node): 
    theta = np.arctan2(p[1] - node.center[1], p[0] - node.center[0]) * 180 / np.pi
    if theta < 90 and theta > 0: 
        node.ur = assignParticle(p, node.ur)
    elif theta < 180 and theta >= 90: 
        node.ul = assignParticle(p, node.ul)
    elif theta < 0 and theta >= -90:
        node.lr = assignParticle(p, node.lr)
    else: # theta < 360 and theta >= 270 
        node.ll = assignParticle(p, node.ll)
    return node
    
        
def assignParticle(p, node):
    # if a leaf has no particle, assign particle to leaf 
    if isinstance(node, leaf) and node.com is None:
        node.com = p
        node.mass = 1
    # if a leaf has a particle, make it a tree and re-assign particles
    elif isinstance(node, leaf) and node.com is not None: 
        p_old = node.com 
        node = QuadTree(node)
        node = assignParticle(p, node)  
        node = assignParticle(p_old, node)
        node.com = (p_old + p) / 2
        node.mass = 2
    # if tree, find out where in the tree the particle should live 
    else: 
        node = quadAssign(p, node)
        node.com = (node.com * node.mass + p) / (node.mass + 1)
        node.mass = node.mass + 1
        
    return node
    

def in_box(com, corners):
    # a point is in a box if it is gt the bottom left corner and lt top right corner 
    return (com[0] >= corners[0][0] and com[0] <= corners[1][0]) and (com[1] >= corners[0][1] and com[1] <= corners[1][1])



def f_multipole(p, node, L, theta=0.5):  
    # check to see if particle is looking at itself,return 0 
    if node.mass is not None and sum((node.com - p)**2)**0.5 < 1e-10:
        return np.zeros(2) 
    
    # if a leaf node and has a particle, directly solve for gravity 
    if isinstance(node, leaf) and node.mass is not None:
        return solve_acc(p, node, L)
    # if a leaf node and doesn't have a particle, return 0
    elif isinstance(node, leaf) and node.mass is None:
        return np.zeros(2)
    
    # if a quadtree that satisfies multipole condition, solve for gravity
    if isinstance(node, QuadTree) and node.length[0] / sum((node.com - p)**2)**0.5 <= theta: 
        return solve_acc(p, node, L)
    # else if quadtree and hasn't satisfied, recursively search the tree's children 
    elif isinstance(node, QuadTree) and node.length[0] / sum((node.com - p)**2)**0.5 > theta: 
        return sum(np.array([f_multipole(p, ch, L, theta=theta) for ch in [node.ll, node.ur, node.lr, node.ul]]))
        

def leapfrog(r, t_start=0, t_end=10, N=1e4, L=2, theta=0.5, multi=False):
    dt = (t_end - t_start)/N

    tpoints = np.arange(t_start, t_end, dt)
    xpoints = []
    trees = []
    
    acc = np.zeros((len(r), 2))
    pos = r[:, 0:2]
    vel = r[:, 2: ]
    
    n_cpu = mp.cpu_count()
    pool = mp.Pool(processes=n_cpu)

    for t in tqdm(tpoints):
        # have to .copy() because we are updating the same array and that causes memory issues 
        xpoints.append(np.hstack((pos, vel, acc)))
        
        # use current acceleration to kick velocity a half time step
        vel = vel + acc * dt / 2 
        # drift the position using the kicked (half time step) velocity 
        pos = pos + vel * dt 
        
        # update acceleration block. construct the quadtree for force modeling 
        root = leaf()
        # which we then can immediately turn into a tree 
        for p in pos: 
            root = assignParticle(p, root)
        trees.append(root)

        # calculate acceleration 
        
#         drawTree(AX, root)
        # multiprocessing is good for large N (> 1000), can be optimized 
        # if multi: 
#         acc = np.vstack([pool.apply_async(f_multipole, args=(p, root, 0)).get() for p in pos])
        # else: 
        acc = np.array([f_multipole(p, root, root.length[0], theta=theta) for p in pos])
        
        # kick velocity a half time step using new updated acceleration  
        vel = vel + acc * dt / 2 
        
        # fix positions based on boundary conditions 
        pos[pos[:, 0] > L/2, 0] = pos[pos[:, 0] > L/2, 0] - L
        pos[pos[:, 0] < -L/2, 0] = pos[pos[:, 0] < -L/2, 0] + L
        pos[pos[:, 1] > L/2, 1] = pos[pos[:, 1] > L/2, 1] - L
        pos[pos[:, 1] < -L/2, 1] = pos[pos[:, 1] < -L/2, 1] + L
        
        
    pool.close()
    pool.join()
        
    return tpoints, np.array(xpoints), np.array(trees)



In [285]:
def subnode(bbox=np.array([[-1, 1],[-1, 1]]), parent_area=1, parent_mass=1): 
    n = leaf(bbox=bbox )    
    n.com = n.center 
    n.mass = parent_mass * (n.length[0]*n.length[1]) / parent_area
    
    return n
    

In [286]:

def solve_acc(p, node, L, epsilon=1e-3, G = 1e-6, m_scale=1e3, r_scale=1): 

#     plt.scatter(p[0], p[1], c='yellow', marker='*', zorder=10)
    # create the 'new' box surrounding the particle 
    bound_corners = np.array([[p[0] - L/2, p[1] - L/2],
                              [p[0] + L/2, p[1] - L/2],
                              [p[0] + L/2, p[1] + L/2],
                              [p[0] - L/2, p[1] + L/2]])
    
    box = np.vstack((bound_corners[0], bound_corners[1], bound_corners[2], 
                     bound_corners[3], bound_corners[0]))
    
#     plt.plot(box[:, 0], box[:, 1], c='white', ls='--')
    
    n_com = copy.copy(node.com )
    n_c = copy.copy(node.center)
    n_l = copy.copy(node.length)
    n_m = copy.copy(node.mass)
    
    # check how many corners are in the effective interaction volume 
    t = [not in_box(np.array([n_c[0] - n_l[0]/2, n_c[1] - n_l[1]/2]), bound_corners[[0, 2]]), 
          not in_box(np.array([n_c[0] + n_l[0]/2, n_c[1] - n_l[1]/2]), bound_corners[[0, 2]]),
          not in_box(np.array([n_c[0] + n_l[0]/2, n_c[1] + n_l[1]/2]), bound_corners[[0, 2]]),
          not in_box(np.array([n_c[0] - n_l[0]/2, n_c[1] + n_l[1]/2]), bound_corners[[0, 2]])]
    
    # if not all or no corners are in, it straddles the boundary and must be subdivided
    # this is the subdivision block. hard to change ... 
    if sum(t) < 4 and sum(t) > 0: 
#         return np.array([0, 0])

#         plotter(AX, node, 'orange', alpha=0.5, zorder=3, ls='-.')
        
        # find the minimum distance to the cell walls
        lx = min(abs(np.array([n_l[0]/2 - (n_c[0] - n_com[0]), n_l[0]/2 + (n_c[0] - n_com[0]) ])))
        ly = min(abs(np.array([n_l[1]/2 - (n_c[1] - n_com[1]), n_l[1]/2 + (n_c[1] - n_com[1]) ])))
        
        # establish the boundaries of the new 'cell' with evenly distributed mass 
        # re_corners = np.array([[n_com[0] - lx, n_com[1] - ly], [n_com[0] + lx, n_com[1] + ly]])

        # establish the boundaries of the new 'cell' with evenly distributed mass 
        re_corners = np.array([[n_com[0] - lx, n_com[1] - ly], 
                               [n_com[0] - lx, n_com[1] + ly],
                               [n_com[0] + lx, n_com[1] + ly],
                               [n_com[0] + lx, n_com[1] - ly]])
        
#         box = np.vstack((re_corners[0], re_corners[1], re_corners[2], re_corners[3], re_corners[0]))
#         plt.plot(box[:, 0], box[:, 1], c='orange', ls='--', alpha=0.25)
        
        # check how many corners are in the effective interaction volume 
        t = [in_box(corner, bound_corners[[0, 2]]) for corner in re_corners]
        
        # if one corner in the volume, need to subdivide into four cells 
        if sum(t) == 1: 
            # find which corner is in the node so we can use it to split it
            b_corner = bound_corners[[in_box(corner, re_corners[[0, 2]]) for corner in bound_corners]][0]
            
            n1 = subnode(bbox=np.array([re_corners[0], b_corner - 1e-9]), 
                        parent_mass=n_m, parent_area=4 * lx * ly)
            n2 = subnode(bbox=np.array([b_corner + 1e-9, re_corners[2]]), 
                        parent_mass=n_m, parent_area=4 * lx * ly)
            n3 = subnode(bbox=np.array([[b_corner[0] + 1e-9, re_corners[0][1] ], 
                                       [re_corners[2][0], b_corner[1] - 1e-9]]), 
                        parent_mass=n_m, parent_area=4 * lx * ly)
            n4 = subnode(bbox=np.array([[re_corners[0][0], b_corner[1] + 1e-9], 
                                        [b_corner[0] - 1e-9, re_corners[2][1]]]), 
                        parent_mass=n_m, parent_area=4 * lx * ly)
            
            
#             [plt.scatter(n_.center[0], n_.center[1], c='orange', alpha=0.5, marker='+') for n_ in [n1, n2, n3, n4]]
#             [plotter(AX, n_, 'orange', alpha=0.5, zorder=3, ls='-.') for n_ in [n1, n2, n3, n4]]
            return np.vstack([solve_acc(p, n_, L) for n_ in [n1, n2, n3, n4]]).sum(axis=0)


            
        # if two corners, need to split into half 
        elif sum(t) == 2:
            # establish empty leaves to hold new nonsense 
            n1 = leaf() 
            n2 = leaf() 
            
            # node is on the right edge of the new volume 
            if t[0] and t[1]:
                n1 = subnode(bbox=np.array([re_corners[0], [bound_corners[2][0] - 1e-9, re_corners[1][1]]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
                n2 = subnode(bbox=np.array([[bound_corners[2][0] + 1e-9 , re_corners[0][1]], re_corners[2]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
            # node is on the left edge of the new volume 
            elif t[2] and t[3]:
                n1 = subnode(bbox=np.array([re_corners[0], [bound_corners[0][0] - 1e-9, re_corners[1][1]]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
                n2 = subnode(bbox=np.array([[bound_corners[0][0] + 1e-9 , re_corners[0][1]], re_corners[2]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
            # node is on the top edge of the new volume 
            elif t[0] and t[3] :               
                n1 = subnode(bbox=np.array([re_corners[0], [re_corners[2][0] , bound_corners[2][1] - 1e-9 ]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
                n2 = subnode(bbox=np.array( [[re_corners[0][0], bound_corners[2][1] + 1e-9], re_corners[2]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
            # elif t[1] and [2]: node is on bottom edge of new volume 
            else:            
                n1 = subnode(bbox=np.array([re_corners[0], [re_corners[2][0] , bound_corners[0][1] - 1e-9 ]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
                n2 = subnode(bbox=np.array( [[re_corners[0][0], bound_corners[0][1] + 1e-9], re_corners[2]]), 
                            parent_mass=n_m, parent_area=4 * lx * ly)
#             [plt.scatter(n_.center[0], n_.center[1], c='orange', alpha=0.5, marker='+') for n_ in [n1, n2]]

            return np.vstack([solve_acc(p, n_, L) for n_ in [n1, n2]]).sum(axis=0)
                
    
    # check if the node's center of mass is within the box. if it is, return standard force calculation 
    # you technically do not need this code block but keep it for now 
    # if sum(t) == 0: 
    if in_box(n_com, bound_corners[[0, 2]]) :
#         plt.plot(node.com[0], node.com[1], c='cyan', marker='+')
        return G * m_scale * node.mass * (n_com - p) / sum((n_com - p)**2 + epsilon)**(3 / 2)
    
    # now we know for sure it's not in the effective volume. we need to then see if the node's box overlaps with our periodic boundary
    # we can check this by seeing if any of the node box's corner's are in the boundary. 
#     plotter(AX, node, 'lime', alpha=0.25, ls='--')

    # we determine the node boundary is outside periodic boundary thus we need to figure out where to loop 
    n_c[n_com > p + L/2] = n_c[n_com > p + L/2] - L
    n_c[n_com < p -L/2] = n_c[n_com < p + -L/2] + L
    
    n_com[n_com > p + L/2] = n_com[n_com > p + L/2] - L
    n_com[n_com < p -L/2] = n_com[n_com < p + -L/2] + L
    

    s = leaf()
    s.com = copy.copy(n_com)
    s.center = copy.copy(n_c)
    s.mass = copy.copy(node.mass )
    s.length = copy.copy(node.length)
#     plotter(AX, s, 'lime', alpha=0.25, ls = '-')
    
    return solve_acc(p, s, L) # G * m_scale * node.mass * (n_com - p) / sum((n_com - p)**2 + epsilon)**(3 / 2)


In [287]:
# some constants ... 
L = 2
n = 10
m = 1
theta = 0 
softening = 0.01

In [288]:
plt.show()

In [ ]:
n = 200
particles = np.random.random((n, 4)) * 2 - 1
# particles[0, 0] = -0.8
# particles[0, 1] = -0.9
# particles1 = np.random.normal(0.6, 0.1, (n, 4))
# particles2 = np.random.normal(-0.6, 0.1, (n, 4))

# particles = np.vstack((particles1, particles2))
particles[:, 2:] = 0
particles[:, 2:] = 0

# FIG, AX = plt.subplots(1, 1, figsize=(6, 6))
# AX.set_facecolor("black")

tpoints, particles_list, trees = leapfrog(particles.copy(), t_start=0, t_end=10, N=1e3, L=2, theta=0.5)
# plt.scatter(particles[:, 0], particles[:, 1], c='white', s=1)

# plotter(AX, trees[0], 'white', alpha=1, zorder=99)


# plt.xlim(-2.05, 1.05)
# plt.ylim(-2.05, 1.05)
plt.show()

 29%|██████████████████████████████████████████████▎                                                                                                                  | 288/1000 [01:58<04:34,  2.60it/s]

In [ ]:

tstart = time.time() 

print('[{:.2f}] Creating plot directory ...'.format(time.time() - tstart))
plot_dir = '{}/{}'.format(os.path.abspath(os.getcwd()), 'L{}n{}'.format(trees[0].length[0], n))
if not os.path.isdir(plot_dir):
    os.mkdir(os.path.join(plot_dir))

print('[{:.2f}] Initializing multiprocessing ...'.format(time.time() - tstart))
n_cpu = mp.cpu_count()
pool = mp.Pool(processes=n_cpu)

mp_trees = np.array_split(trees, n_cpu)
mp_particles = np.array_split(particles_list, n_cpu)
mp_tpoints = np.array_split(np.arange(len(tpoints)), n_cpu)
    
print('[{:.2f}] Creating plots ...'.format(time.time() - tstart))
for i in range(n_cpu):
    pool.apply_async(renderParticles, args=(mp_particles[i], mp_trees[i], mp_tpoints[i], plot_dir))

pool.close()
pool.join()
print('[{:.2f}] Multiprocess concluded ...'.format(time.time() - tstart))


print('[{:.2f}] Creating video ...'.format(time.time() - tstart))

fps=30 #number of frames per second
image_files = natsorted([os.path.join(plot_dir,img) for img in os.listdir(plot_dir) if img.endswith(".png")], reverse=False)
clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
clip.write_videofile('test.mp4')

print('[{:.2f}] Video created ...'.format(time.time() - tstart))

_ = [os.remove(image_file) for image_file in image_files]
os.rmdir(plot_dir)
print('[{:.2f}] Directory cleaned ...'.format(time.time() - tstart))


In [ ]:

# def solve_acc(p, node, L, epsilon=1e-3, G = 1e-6, m_scale=1e3, r_scale=1): 
#     # create the 'new' box surrounding the particle 
#     bound_corners = np.array([[p[0] - L/2, p[1] - L/2],
#                               [p[0] + L/2, p[1] - L/2],
#                               [p[0] + L/2, p[1] + L/2],
#                               [p[0] - L/2, p[1] + L/2]])
    
#     n_com = node.com 
#     n_c = node.center
#     n_l = node.length
#     n_m = node.mass
    
#     # check how many corners are in the effective interaction volume 
#     t = [not in_box(np.array([n_c[0] - n_l[0]/2, n_c[1] - n_l[1]/2]), bound_corners[[0, 2]]), 
#           not in_box(np.array([n_c[0] + n_l[0]/2, n_c[1] - n_l[1]/2]), bound_corners[[0, 2]]),
#           not in_box(np.array([n_c[0] + n_l[0]/2, n_c[1] + n_l[1]/2]), bound_corners[[0, 2]]),
#           not in_box(np.array([n_c[0] - n_l[0]/2, n_c[1] + n_l[1]/2]), bound_corners[[0, 2]])]
    
#     # if not all or no corners are in, it straddles the boundary and must be partitioned
#     if sum(t) < 4 and sum(t) > 0:         
#         # find the minimum distance to the cell walls
#         lx = min(abs(np.array([n_l[0]/2 - (n_c[0] - n_com[0]), n_l[0]/2 + (n_c[0] - n_com[0]) ])))
#         ly = min(abs(np.array([n_l[1]/2 - (n_c[1] - n_com[1]), n_l[1]/2 + (n_c[1] - n_com[1]) ])))
        
#         # establish the boundaries of the new 'cell' with evenly distributed mass 
#         re_corners = np.array([[n_com[0] - lx, n_com[1] - ly], 
#                                [n_com[0] - lx, n_com[1] + ly],
#                                [n_com[0] + lx, n_com[1] + ly],
#                                [n_com[0] + lx, n_com[1] - ly]])
        
#         # check how many corners are in the effective interaction volume 
#         t = [in_box(corner, bound_corners[[0, 2]]) for corner in re_corners]
        
#         # if one corner in the volume, need to subdivide into four cells 
#         if sum(t) == 1: 
#             corner1 = re_corners[t]
#             corner2 = bound_corners[[in_box(corner, re_corners[[0, 2]]) for corner in bound_corners]]
            
#             return np.array([0, 0])

            
#         # if two corners, need to split into half 
#         elif sum(t) == 2:
#             # establish empty leaves to hold new nonsense 
#             n1 = leaf() 
#             n2 = leaf() 
            
#             # node is on the right edge of the new volume 
#             if t[0] and t[1]:
#                 n1 = leaf(bbox=np.array([re_corners[0], [bound_corners[2][0] - 1e-10, re_corners[1][1]]]))    
#                 n1.com = n1.center
#                 n1.mass = n_m * (n1.length[0]*n1.length[1]) / (4 * lx * ly)
                
#                 n2 = leaf(bbox=np.array([[bound_corners[2][0] + 1e-10 , re_corners[0][1]], re_corners[2]] ))    
#                 n2.com = n2.center 
#                 n2.mass = n_m * (n2.length[0]*n2.length[1]) / (4 * lx * ly)
                
#             # node is on the left edge of the new volume 
# #             elif t[2] is True and t[3] is True:
                
#             # node is on the top edge of the new volume 
#             elif t[0] and t[3] :               
#                 n1 = leaf(bbox=np.array([re_corners[0], [re_corners[2][0] , bound_corners[2][1] - 1e-10 ] ]))    
#                 n1.com = n1.center
#                 n1.mass = n_m * (n1.length[0]*n1.length[1]) / (4 * lx * ly)
                
#                 n2 = leaf(bbox=np.array( [[re_corners[0][0], bound_corners[2][1] + 1e-10], re_corners[2]]     ))    
#                 n2.com = n2.center 
#                 n2.mass = n_m * (n2.length[0]*n2.length[1]) / (4 * lx * ly)
# #             # elif t[1] is True and t[2] is True: but we know this is the only other situation
# #             else: 
        
#             return np.array([solve_acc(p, n_, L) for n_ in [n1, n2]]).sum(axis=1)
                

#         # if no corners, we can assume it's completely inside or outside the boundary 
#         else: 
#             return solve_acc(p, s, L) 
    
#     # check if the node is fully within the box. if it is, return standard force calculation 
#     # you technically do not need this code block but keep it for now 
#     if in_box(n_com, bound_corners[[0, 2]]) :
#         return G * m_scale * node.mass * (n_com - p) / sum((n_com - p)**2 + epsilon)**(3 / 2)
    
#     # now we know for sure it's not in the effective volume. we need to then see if the node's box overlaps with our periodic boundary
#     # we determine the node boundary is outside periodic boundary thus we need to figure out where to loop 
#     n_c[n_com > p + L/2] = n_c[n_com > p + L/2] - L
#     n_c[n_com < p -L/2] = n_c[n_com < p + -L/2] + L
    
#     n_com[n_com > p + L/2] = n_com[n_com > p + L/2] - L
#     n_com[n_com < p -L/2] = n_com[n_com < p + -L/2] + L
    
#     s = leaf()
#     s.com = n_com
#     s.center = n_c
#     s.mass = node.mass 
#     s.length = node.length
    
#     # need to recursively solve in case periodic boundary leads to node landing on edge 
#     return solve_acc(p, s, L) 